In [1]:
import os

from d_2021.config.path import *
from d_2021.utils.import_utils import *
from d_2021.utils.math_utils import *
from d_2021.utils.common_utils import *

from tqdm import *
import pandas as pd

# 第一步：数据清洗

In [30]:
def data_pre_threat():
    if os.path.exists(md_df_path):
        md_df_3 = pd.read_csv(md_df_path)
        md_names = md_df_3.columns.tolist()
    else:
        md_names, md_df = get_objs_in_csv("../dataset/train/Molecular_Descriptor.csv")
        print('总特征数:', len(md_names) - 1)
        error_names = []
        # 使用箱线法去除异常值出现大于100次的列
        for md_name in md_names[1:]:
            row = md_df[md_name].tolist()
            # row = str_list_2_int(row)
            min_num, Q1, Q2, Q3, max_num = get_box_plot(row, md_name, False)
            # 计算异常值数量
            error_count = 0
            for i in range(len(row)):
                cell = md_df[md_name][i]
                if cell < min_num:
                    error_count += 1
                    md_df.loc[i, md_name] = min_num
                if cell > max_num:
                    error_count += 1
                    md_df.loc[i, md_name] = max_num
            if error_count > 100:
                error_names.append(md_name)
        print('异常值数量:', len(error_names))
        md_df_2 = remove_names(md_df, md_names, error_names)
        # 含0大于0.9的列
        zero_names = []
        # 计算列内含0大于90%的列
        for md_name in md_names[1:]:
            row = md_df_2[md_name]
            p = count_zero(row)
            if p > 0.9:
                zero_names.append(md_name)
        print('0值大于90%数量:', len(zero_names))
        md_df_3 = remove_names(md_df_2, md_names, zero_names)
        print('筛选后总数', len(md_names) - 1)
        md_df_3.to_csv(md_df_path, index=False)
    return md_df_3, md_names

In [31]:
pre_threat_df, pre_threat_names = data_pre_threat()
pre_threat_df

总特征数: 729
异常值数量: 155
0值大于90%数量: 327
筛选后总数 247


,SMILES,ALogP,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nC,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCC3)c4ccc(OCCN5C...,-0.2860,126.1188,74.170169,12.0,12.0,64.0,31.0,33.0,26.0,...,439.218115,64.771680,2.089409,15.471445,8.858910,3.406628,3011.0,47.0,4.666,166.0
1,Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCCCC3)c4ccc(OCCN...,-0.8620,131.9420,80.357341,12.0,12.0,70.0,33.0,37.0,28.0,...,467.249415,68.960024,2.089698,15.486947,8.863774,3.406648,3516.0,54.0,5.804,174.0
2,Oc1ccc(cc1)[C@H]2Sc3cc(O)ccc3O[C@H]2c4ccc(OCCN...,0.7296,139.9304,74.064997,18.0,18.0,62.0,33.0,29.0,27.0,...,463.181729,68.748923,2.083301,18.011114,11.390412,3.406644,3542.0,52.0,2.964,176.0
3,Oc1ccc2O[C@H]([C@@H](CC3CCCCC3)Sc2c1)c4ccc(OCC...,-0.3184,133.4822,80.357341,12.0,12.0,70.0,33.0,37.0,28.0,...,467.249415,68.883696,2.087385,15.468365,8.857943,3.406624,3594.0,50.0,6.015,174.0
4,Oc1ccc2O[C@H]([C@@H](Cc3ccccc3)Sc2c1)c4ccc(OCC...,1.3551,143.1903,76.356583,18.0,18.0,64.0,33.0,31.0,28.0,...,461.202465,68.883696,2.087385,15.468365,8.857943,3.406624,3594.0,50.0,4.462,174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,COc1cc(OC)cc(\C=C\c2cccc(OS(=O)(=O)C3CC4OC3C(=...,1.8193,177.6817,89.159790,24.0,24.0,73.0,43.0,30.0,34.0,...,598.166139,88.709996,2.063023,25.470481,20.590936,0.000000,7121.0,70.0,2.526,236.0
1970,Oc1ccc(cc1)C2=C(C3OC2CC3S(=O)(=O)Oc4cccc(\C=C\...,1.6903,167.6057,82.972618,24.0,24.0,67.0,41.0,26.0,32.0,...,570.134839,84.662088,2.064929,24.928962,20.590936,0.000000,6171.0,66.0,1.884,228.0
1971,Oc1ccc(cc1)C2=C(C3OC2CC3S(=O)(=O)Oc4ccc(\C=C\c...,1.6903,167.6057,82.972618,24.0,24.0,67.0,41.0,26.0,32.0,...,570.134839,84.660642,2.064894,24.923083,20.590936,0.000000,6421.0,66.0,1.884,228.0
1972,Oc1ccc(cc1)C2=C([C@H]3O[C@H]2C[C@@H]3S(=O)(=O)...,1.3365,125.5605,63.287860,18.0,18.0,51.0,31.0,20.0,24.0,...,436.098059,64.171346,2.070043,19.841924,16.326873,0.000000,2583.0,50.0,0.782,174.0


# 第二步：灰色关联度分析

In [4]:
def get_sequence(m_df, names, csv_path):
    """
    得到计算好的序列
    :param m_df: 数据表
    :param names: 名称
    :param csv_path: 路径
    :return: 处理好的序列
    """
    if os.path.exists(csv_path):
        sequence = pd.read_csv(csv_path)
    else:
        sequence = get_sequence_divide_mean(m_df, names)
        sequence.to_csv(csv_path, index=False)
    return sequence

In [5]:
def get_mean_sequence():
    era_names, era_df = get_objs_in_csv("../dataset/train/ERa_activity.csv")
    # 得到母序列均值商
    parent = get_sequence(era_df, ['pIC50'], csv1_path)
    # 得到子序列均值商
    child_names1 = pre_threat_names[1:]
    child = get_sequence(pre_threat_df, child_names1, csv2_path)
    return parent, child, child_names1

## 指标正向化，确定分析数列，变量预处理

In [6]:
parent_sequence, child_sequence, child_names = get_mean_sequence()

In [7]:
def cal_ab():
    local_min = 100
    local_max = 0
    for name in pre_threat_names[1:]:
        # 对每一个子序列，计算对应值
        sequence = child_sequence[name]
        for i in range(len(sequence)):
            # 每一行的数相减
            t = abs(child_sequence.iloc[i] - parent_sequence['pIC50'][i])
            if t < local_min:
                local_min = t
            if t > local_max:
                local_max = t
    return local_min, local_max

In [8]:
a, b = cal_ab()

## 计算子序列中各个指标与母序列的关联系数

In [9]:
p = 0.5

In [10]:
def cal_relevancy():
    copy_child = child_sequence.copy()
    if os.path.exists(csv3_path):
        copy_child = pd.read_csv(csv3_path)
    else:
        for name in tqdm(child_names):
            # 对每一个子序列，计算对应值
            sequence = copy_child[name]
            for i in range(len(sequence)):
                # 计算关联度
                sequence[i] = (a + p * b) / (abs(sequence[i] - parent_sequence['pIC50'][i]) + p * b)
        copy_child.to_csv(csv3_path, index=False)
    return copy_child

In [11]:
relevancy_df = cal_relevancy()

In [12]:
def cal_relevancy_mean(m_df: pd.DataFrame):
    obj_list = {}
    for name in tqdm(child_names):
        # 对每一个子序列，计算平均值
        m_mean = m_df[name].mean()
        obj_list[name] = m_mean
    # t_df = pd.DataFrame(obj_list)
    return obj_list

In [13]:
relevancy_mean_df = cal_relevancy_mean(relevancy_df)

100%|██████████| 247/247 [00:00<00:00, 35287.07it/s]


In [14]:
t_relevancy_mean_df = sorted(relevancy_mean_df.items(), key=lambda x: x[1], reverse=True)

获取灰色关联度前100个

In [22]:
gray_name = []
for t in t_relevancy_mean_df:
    gray_name.append(t[0])

In [23]:
gray_name = gray_name[:100]

In [24]:
forest_dataset = pre_threat_df[gray_name]
forest_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 100 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BCUTp-1h           1974 non-null   float64
 1   MLogP              1974 non-null   float64
 2   VAdjMat            1974 non-null   float64
 3   maxaaCH            1974 non-null   float64
 4   ETA_Psi_1          1974 non-null   float64
 5   WTPT-2             1974 non-null   float64
 6   ETA_Epsilon_3      1974 non-null   float64
 7   PetitjeanNumber    1974 non-null   float64
 8   maxwHBa            1974 non-null   float64
 9   ETA_BetaP_s        1974 non-null   float64
 10  ETA_Epsilon_5      1974 non-null   float64
 11  ETA_Epsilon_2      1974 non-null   float64
 12  AMR                1974 non-null   float64
 13  SP-2               1974 non-null   float64
 14  Zagreb             1974 non-null   float64
 15  SP-3               1974 non-null   float64
 16  CrippenMR          1974

In [25]:
forest_dataset = forest_dataset.apply(lambda row: row * 10000, axis=1)

In [26]:
forest_dataset

,BCUTp-1h,MLogP,VAdjMat,maxaaCH,ETA_Psi_1,WTPT-2,ETA_Epsilon_3,PetitjeanNumber,maxwHBa,ETA_BetaP_s,...,ETA_Eta_L,nAromBond,Kier2,LipoaffinityIndex,naasC,SPC-6,ATSm1,VP-4,ATSp5,MAXDP2
0,132548.5902,37700.0,59541.96310,22228.15571,6175.7,20894.09041,4458.8,4705.88235,22228.15571,6290.3,...,99932.0,120000.0,109505.20830,118309.49200,50000.0,90197.29143,398111.4045,67300.82481,37927797.60,39215.40668
1,136176.0896,39900.0,60443.94119,22384.40571,6227.6,20896.97692,4450.5,4705.88235,22384.40571,6212.1,...,109932.0,120000.0,123008.00000,131499.73910,50000.0,92554.31403,418111.4045,70836.35871,41543847.41,40115.40668
2,133934.5285,37700.0,60443.94119,21171.21914,5987.6,20833.00711,4450.5,4705.88235,21171.21914,6287.9,...,86902.4,180000.0,118231.44940,105771.77300,70000.0,97607.36758,425856.0948,57865.64953,39937340.54,39301.00601
3,130795.4735,39900.0,60443.94119,22286.87484,6227.6,20873.84716,4450.5,4705.88235,22286.87484,6212.1,...,109763.6,120000.0,123008.00000,132127.66890,50000.0,87060.54056,418111.4045,70108.24635,38501901.06,39776.21056
4,130434.5358,39900.0,60443.94119,22089.95777,6227.6,20873.84716,4450.5,4705.88235,22089.95777,6212.1,...,89941.8,180000.0,123008.00000,124896.90710,60000.0,87060.54056,418111.4045,59902.39144,38061206.73,39459.34813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,127753.1495,42100.0,64262.64755,18729.73079,5616.4,20630.23171,4445.4,5000.00000,18890.80073,6279.1,...,90420.3,240000.0,144085.71430,102929.31560,90000.0,179511.51910,553235.0532,59693.29376,62376298.55,66989.68018
1970,127740.8648,39900.0,63575.52005,17891.47690,5555.5,20649.28967,4451.3,4736.84211,17891.47690,6219.5,...,84543.3,240000.0,131574.39450,78840.91608,90000.0,174918.95450,533235.0532,57579.21020,59485337.03,66050.74198
1971,127291.9207,39900.0,63575.52005,17842.93583,5555.5,20648.93717,4451.3,5000.00000,17842.93583,6219.5,...,84543.3,240000.0,131574.39450,79650.74063,90000.0,174123.29240,533235.0532,57242.16100,58040105.90,65111.68390
1972,126842.9911,33300.0,59541.96310,18038.75581,5583.4,20700.43427,4458.8,5000.00000,18038.75581,6290.3,...,66589.8,180000.0,93306.21302,62365.58223,50000.0,145078.97740,417745.6725,46590.60531,47275568.04,61130.40120


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

x, y = forest_dataset.iloc[:, 1:].values, forest_dataset.iloc[:, 0].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
feat_labels = forest_dataset.columns[1:]
forest = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)

In [28]:
forest.fit(x_train, y_train.astype('int'))

RandomForestClassifier(n_jobs=-1, random_state=0)

In [29]:
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) MAXDP2                         0.015986
 2) gmax                           0.015797
 3) BCUTp-1l                       0.015075
 4) maxHBa                         0.014967
 5) LipoaffinityIndex              0.014728
 6) maxaasC                        0.014603
 7) ETA_EtaP_B_RC                  0.014440
 8) WTPT-2                         0.014122
 9) CrippenLogP                    0.014113
10) ETA_Shape_Y                    0.013895
11) minaaCH                        0.013806
12) minHother                      0.013334
13) MDEC-23                        0.013321
14) MLFER_S                        0.013290
15) maxwHBa                        0.013283
16) maxaaCH                        0.013005
17) maxHaaCH                       0.012977
18) ATSp5                          0.012949
19) maxHother                      0.012861
20) MLFER_E                        0.012588
21) ATSp4                          0.012532
22) ETA_EtaP_F_L                   0.012502
23) ETA_EtaP_L                  